# Sampling Raster Data using Points 

In [2]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import os
import numpy as np
import rasterio as rasterio
from rasterio.plot import show
import richdem as rd
import cv2 as cv

In [3]:
#normalization function
def normalize(x):
    num= x- x.min()
    den= x.max()-x.min()
    return num/den

In [ ]:
#Sample 50 landslide locations and 50 non-landslide locations from each image.
np.random.seed(123)#seed

coord_landslide_1= [] #landslide
coord_landslide_0= [] #no landslide
blank_img= np.zeros(shape=(128, 128)) #for retrieving the mask
da_landslide= pd.DataFrame(columns= {"p/a", "x", "y","img"}) #dataframe to store the extracted values

base_folder = r'C:\Users\serra\OneDrive\Documents\DATA\train' #set the path of the base folder where we have the img and the masks
img_folder = base_folder + "\\img" #img path
mask_folder = base_folder + "\\mask" #mask path
for imagen_file in glob.glob(img_folder + "\\*.h5"): #For each set of image and mask...
    imagen_dt = h5py.File(imagen_file, 'r') #load the image
    images = imagen_dt['img']
    img_file_name = os.path.basename(imagen_file)
    img_name = img_file_name.split(".")[0] #set image name
    mask_file = mask_folder + "\\" + img_file_name.replace("image","mask") #load the mask
    mask_dt = h5py.File(mask_file, 'r')
    masks = mask_dt['mask']
    
    #Extract, denoise and calculate the useful bands from the img
    dem= images[:,:,13] #extract dem from the 13th band
    dem= normalize(dem) #normalize dem
    slope= images[:,:,12] #extract slope from the 12th band
    slope= normalize(slope) #normalize slope
    k = 5 #smoothing factor
    dem_suavizado = cv.blur(dem,(k,k)) #reducción de ruido dem
    slope_suavizado = cv.blur(slope,(k,k)) #reducción de ruido slope
    dem_richdem = rd.rdarray(dem_suavizado, no_data=-9999) #convert to rdarray in order to generate other bands from the dem
    dem_curvature = rd.TerrainAttribute(dem_richdem, attrib="curvature") #calculate curvature
    dem_curvature= normalize(dem_curvature)#normalize curvature
    dem_planform_curvature = rd.TerrainAttribute(dem_richdem, attrib="planform_curvature") #calculate planform curvature
    dem_planform_curvature= normalize(dem_planform_curvature) #normalize planform curvature
    dem_profile_curvature = rd.TerrainAttribute(dem_richdem, attrib="profile_curvature") #calculate profile curvature
    dem_profile_curvature= normalize(dem_profile_curvature) #normalize profile curvature
    NDVI= (images[:,:,7]-images[:,:,3])/(images[:,:,7]+images[:,:,3]) #calculate NDVI
    NDVI= normalize(NDVI) #normalize NDVI
    BSI= ((images[:,:,10] + images[:,:,3]) - (images[:,:,7] + images[:,:,1]) / (images[:,:,10] + images[:,:,3]) + (images[:,:,7] + images[:,:,1])) #calculate BSI
    BSI= normalize(BSI) #normalize BSI
    bands= np.array([dem_suavizado, slope_suavizado, dem_curvature, dem_planform_curvature, dem_profile_curvature, NDVI, BSI]) #make an array with all the bands
    
    
    if np.count_nonzero(masks) >= 50: #If there are more landslide pixels than 50, select 50 locations
        l = 50
    else:
        l = np.count_nonzero(masks) # If there are less than 50 landslide pixels, select that number of locations 
    if l==0: continue #If there are no landslides then skip
    
    new_img_np_0= np.ma.array(blank_img, mask=masks) #masked image for non-landslide areas
    xt,yt = np.where(new_img_np_0.mask <=False)  # select only non landslide values
    for j in range(l):
        k= np.random.randint(len(xt))  # select random locations
        random_pos = [xt[k], yt[k]]  # random locations
        coord_landslide_0.append(random_pos)  # list of coordinates
        indice = len(da_landslide)
        da_landslide.loc[indice, ["x"]] = random_pos[0]  # save coordinate x to the dataframe 
        da_landslide.loc[indice, ["y"]] = random_pos[1]  # save coordinate y to the dataframe
        for i in range(len(bands)):
            img_np = bands[i, :, :] #for each band
            da_landslide.loc[indice, [i]] = img_np[
                coord_landslide_0[j][0], coord_landslide_0[j][1]]  # extract the values from the coordinates and save them in the dataframe
        da_landslide.loc[indice, ["p/a"]] = 0  # landslide absence (dataframe)
        da_landslide.loc[indice, ["img"]] = img_name  # write image name in the dataframe
    
    new_img_np_1 = np.ma.array(blank_img, mask=np.logical_not(masks)) #masked image for landslide areas
    xf,yf = np.where(new_img_np_1.mask <=False) #select only landslide values
    for j in range(l):
        k = np.random.randint(len(xf))  # select random locations
        random_pos = [xf[k], yf[k]]  # random locations
        coord_landslide_1.append(random_pos)  # list of coordinates
        indice = len(da_landslide)
        da_landslide.loc[indice, ["x"]] = random_pos[0]  # save coordinate x to the dataframe
        da_landslide.loc[indice, ["y"]] = random_pos[1]  # save coordinate y to the dataframe
        for i in range(len(bands)):
            img_np = bands[i, :, :] #for each band
            da_landslide.loc[indice, [i]] = img_np[
                coord_landslide_1[j][0], coord_landslide_1[j][1]]  # extract the values and save them in the table
        da_landslide.loc[indice, ["p/a"]] = 1  # landslide presence (dataframe)

da_landslide.to_csv("data_extraction.csv",decimal=".", sep =";") #save the csv